# Tour of the universe

This Jupyter notebook models a tour of the universe at constant 1g acceleration. We explore journeys of between 1 and 25 million light years, and calculate the time experienced by the traveller and the coordinate time (eg on Earth).

In [26]:
# Light years to calculate

distances: list[int] = [
    1,
    2,
    3,
    4,
    5,
    10,
    50,
    100,
    200,
    500,
    # 8, 10, 12, 14, 16, 20, 25,
    # 30, 40, 50, 60, 70, 80, 90,
    # 100, 200, 300, 400, 500,
    1000,
    2000,
    5000,
    10000,
    20000,
    50000,
    100000,
    500000,
    1000000,
    2500000,
    5000000,
    10000000,
    25000000,
]

## Code to calculate the journey

Has option for flip-and-burn at the halfway point, to slow down and return to the start, or to continue accelerating to the destination.

In [27]:
from mpmath import mp
import relativity_lib as rl
from prettytable import PrettyTable  # , TableStyle

# use 100 decimal places
rl.configure(300)
table = PrettyTable(["Ly", "Prop Yrs", "Co'rd Yrs", "Peak c", "Lorentz", "Apparent c"])
table.align = "r"  # right align everything


def process_dist(distance_ly, flip: bool) -> None:
    accel = "1"
    distance = rl.light_year * rl.ensure(distance_ly)
    a = rl.g * mp.mpf(accel)

    if flip:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(
            a, distance / 2
        )  # accelerate half-way

        # convert to years
        burn_years = burn_sec * 2 / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) * 2 / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)
    else:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(a, distance)

        # convert to years
        burn_years = burn_sec / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)

    apparent_speed = (
        rl.ensure(distance_ly) / burn_years
    )  # apparent speed in proper years per light year

    table.add_row(
        [
            rl.format_mpf(distance_ly, 0),
            rl.format_mpf(burn_years, 2),
            rl.format_mpf(coord_years, 1),
            rl.format_mpf_significant(peak_velocity, 2, "9"),
            rl.format_mpf(peak_lorentz, 1),
            rl.format_mpf(apparent_speed, 2),
        ]
    )

## Flip-and-burn results

`Prop yrs` is the proper time experienced by the traveller, `Coord yrs` is the coordinate time on Earth. `Peak c` is the peak speed in terms of the speed of light, `Lorentz` is the Lorentz factor which gives the time dilation and length contraction effects.

`Apparent c` is interesting. Its' the coordinate journey length (uncontracted) divided by proper years experienced by the traveller. Even though the traveller will always measure their speed as less than c (`peak c`), as the lorentz factor causes length contraction the 'apparent' speed can be greater than c. Eg you can travel 25 million light years in 33 years of proper time, which is an apparent speed of 756,097 * c.

This hyper-c apparent velocity is only experienced for the traveller, not for the outside universe.

Using flip-and-burn you can reach the closest stars in about 4 years, and reach the Andromeda galaxy in 28 years.

In [28]:
for distance in distances:
    process_dist(distance, True)

print(table)

+------------+----------+--------------+--------------------+--------------+------------+
|         Ly | Prop Yrs |    Co'rd Yrs |             Peak c |      Lorentz | Apparent c |
+------------+----------+--------------+--------------------+--------------+------------+
|          1 |     1.89 |          2.2 |               0.75 |          1.5 |       0.52 |
|          2 |     2.58 |          3.4 |               0.87 |          2.0 |       0.77 |
|          3 |     3.07 |          4.5 |              0.919 |          2.5 |       0.97 |
|          4 |     3.45 |          5.6 |              0.945 |          3.0 |       1.15 |
|          5 |     3.77 |          6.6 |              0.960 |          3.5 |       1.32 |
|         10 |     4.85 |         11.7 |              0.986 |          6.1 |       2.06 |
|         50 |     7.71 |         51.9 |            0.99930 |         26.8 |       6.48 |
|        100 |     9.02 |        101.9 |            0.99981 |         52.6 |      11.08 |
|        2

## Non-stop (full burn)

Without stopping at the halfway point to slow down, journey time is much reduced. However you wouldn't see much at your destination, which would be passed at `peak c`.

You can reach the closest stars in about 2.5 years, and cross the galaxy in 12 years.

In [29]:
table.clear_rows()
for distance in distances:
    process_dist(distance, False)

print(table)

+------------+----------+--------------+---------------------+--------------+--------------+
|         Ly | Prop Yrs |    Co'rd Yrs |              Peak c |      Lorentz |   Apparent c |
+------------+----------+--------------+---------------------+--------------+--------------+
|          1 |     1.29 |          1.7 |                0.87 |          2.0 |         0.77 |
|          2 |     1.72 |          2.8 |               0.945 |          3.0 |         1.15 |
|          3 |     2.02 |          3.8 |               0.969 |          4.0 |         1.48 |
|          4 |     2.24 |          4.8 |               0.980 |          5.1 |         1.78 |
|          5 |     2.42 |          5.8 |               0.986 |          6.1 |         2.06 |
|         10 |     3.02 |         10.9 |              0.9960 |         11.3 |         3.31 |
|         50 |     4.51 |         50.9 |             0.99981 |         52.6 |        11.08 |
|        100 |     5.17 |        100.9 |            0.999953 |        

## Rapidity

*Rapidity* is useful for working with relativistic velocities. It's additive, so you can add rapidities to get the rapidity of the combined system. It's also a linear function of velocity, so you can add rapidities to get the rapidity of the combined system. Then use the `velocity_from_rapidity` to convert it back.

*added_velocity = velocity(rapidity(v1) + rapidity(v2))*

https://en.wikipedia.org/wiki/Rapidity

In [30]:
rl.configure(200)

# Velocities to add
velocity1 = mp.mpf("299792457.9999876")
velocity2 = mp.mpf("299792457.9999999123")

# convert to rapidity
rapidity1 = rl.rapidity_from_velocity(velocity1)
rapidity2 = rl.rapidity_from_velocity(velocity2)

# add rapidities, and convert back to velocity
added_rapidity = rapidity1 + rapidity2
final_velocity = rl.velocity_from_rapidity(added_rapidity)

# display results
print(f"Velocity 1: {rl.format_mpf(velocity1, -1)} m/s")
print(f"Rapidity 1: {rl.format_mpf(rapidity1, 5)}")
print(f"Velocity 2: {rl.format_mpf(velocity2, -1)} m/s")
print(f"Rapidity 2: {rl.format_mpf(rapidity2, 5)}")
print(f"Added rapidity: {rl.format_mpf(added_rapidity, 5)}")
print(f"Added velocity: {rl.format_mpf_significant(final_velocity, 3, '9')} m/s")

Velocity 1: 299,792,457.9999876 m/s
Rapidity 1: 15.75478
Velocity 2: 299,792,457.9999999123 m/s
Rapidity 2: 18.23054
Added rapidity: 33.98532
Added velocity: 299,792,457.99999999999999999999818 m/s
